In [1]:
import math
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from statsmodels.tools import eval_measures
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [192]:
nflis_data = pd.read_csv("2018_MCMProblemC_DATA/MCM_NFLIS_Data_onePage.csv")

In [194]:
nflis_data.head()

,YYYY,State,COUNTY,FIPS_State,FIPS_County,FIPS_Combined,SubstanceName,DrugReports,TotalDrugReportsCounty,TotalDrugReportsState
0,2010,VA,ACCOMACK,51,1,51001,Propoxyphene,1,84,41462
1,2010,OH,ADAMS,39,1,39001,Morphine,9,527,70999
2,2010,PA,ADAMS,42,1,42001,Methadone,2,334,89981
3,2010,VA,ALEXANDRIA CITY,51,510,51510,Heroin,5,427,41462
4,2010,PA,ALLEGHENY,42,3,42003,Hydromorphone,5,8500,89981


In [200]:
year_lst = []
state_lst = []
county_lst = []
substance_lst = []
substance_count_lst = []
total_count_county = []
total_count_state = []
total_count_county_include_others = []
total_count_state_include_others = []

for year in sorted(set(nflis_data['YYYY'])):
    year_data = nflis_data.loc[nflis_data['YYYY'] == year]

    for state in set(year_data['State']):
        state_data = year_data.loc[year_data['State'] == state]

        for county in set(state_data['COUNTY']):
            county_data = state_data.loc[state_data['COUNTY'] == county]
            
            for substanceName in set(county_data['SubstanceName']):
                substance_data = county_data.loc[county_data['SubstanceName'] == substanceName]
                count = int(substance_data['DrugReports'])
                county_total = substance_data['TotalDrugReportsCounty']
                state_total = substance_data['TotalDrugReportsState']
                
                county_dict[substanceName] = count
                
                year_lst.append(year)
                state_lst.append(state)
                county_lst.append(county)

                substance_lst.append(substanceName)
                substance_count_lst.append(count)
                
                total_count_county_include_others.append(int(county_total))
                total_count_state_include_others.append(int(state_total))
            for i in range(len(set(county_data['SubstanceName']))):
                total_count_county.append(sum(county_data['DrugReports']))
                total_count_state.append(sum(state_data['DrugReports']))

In [223]:
drug_rate_county = []
for i, j in zip(substance_count_lst, total_count_county):
    drug_rate_county.append(i / j)

In [225]:
drug_rate_state = []
for i, j in zip(substance_count_lst, total_count_state):
    drug_rate_state.append(i / j)

In [239]:
drug_rate_county_total = []
for i, j in zip(substance_count_lst, total_count_county_include_others):
    drug_rate_county_total.append(i / j)

In [230]:
drug_rate_state_total = []
for i, j in zip(substance_count_lst, total_count_state_include_others):
    drug_rate_state_total.append(i / j)

In [243]:
update_data = {
    'Year': year_lst,
    'State': state_lst,
    'County': county_lst,
    'SubstanceName': substance_lst,
    'DrugReports': substance_count_lst,
    'DrugReportsCounty': total_count_county,
    'TotalDrugReportsCounty': total_count_county_include_others,
    'DrugReportsState': total_count_state,
    'TotalDrugReportsState': total_count_state_include_others,
    'County_Rate': drug_rate_county,
    'State_Rate': drug_rate_state,
    'County_Rate_Total': drug_rate_county_total,
    'State_Rate_Total': drug_rate_state_total 
}

In [244]:
updated_data = pd.DataFrame.from_dict(update_data)

In [245]:
updated_data.head()

,Year,State,County,SubstanceName,DrugReports,DrugReportsCounty,TotalDrugReportsCounty,DrugReportsState,TotalDrugReportsState,County_Rate,State_Rate,County_Rate_Total,State_Rate_Total
0,2010,VA,PITTSYLVANIA,Methadone,2,16,182,8685,41462,0.1250,0.000230,0.010989,0.000048
1,2010,VA,PITTSYLVANIA,Heroin,1,16,182,8685,41462,0.0625,0.000115,0.005495,0.000024
2,2010,VA,PITTSYLVANIA,Hydrocodone,7,16,182,8685,41462,0.4375,0.000806,0.038462,0.000169
3,2010,VA,PITTSYLVANIA,Oxycodone,5,16,182,8685,41462,0.3125,0.000576,0.027473,0.000121
4,2010,VA,PITTSYLVANIA,Morphine,1,16,182,8685,41462,0.0625,0.000115,0.005495,0.000024


In [246]:
updated_data.to_csv("2018_MCMProblemC_DATA/updated_nflis_data.csv", index=False)